In [2]:
#Step 1: align genomes externally.

#Step 2: Verify that genome alignment looks good.

#Step 3: Run R code below.  Note you need to run the code 2x to convert the "raw" model to actual nucleotide differences.

library(ape)
library(phangorn, lib.loc='/scicomp/home-pure/evk3/R/x86_64-pc-linux-gnu-library/4.0')

#Align genomes!!

#Read in aligned EBOV genomes:
alignment<- read.dna("./finished_genomes_SUD_13Feb2023_no-dates_aln.fasta", format="fasta")

#To count number of unknown sites:
table(as.character(alignment))

#Remove NA sites:
na.posi <- which(apply(as.character(alignment),2, function(e) any(!e %in% c("a","t","g","c"))))
alignment_no_NAs <- alignment[,-na.posi]
table(as.character(alignment_no_NAs))

                       
alignment
alignment_no_NAs

#Calculate genetic distance with raw differences, no with the TN93 model:
dst <- dist.dna(alignment_no_NAs, model="raw", as.matrix=TRUE)

#Change the value below based on code output to convert raw values to nucleotide differences.
dst*13503
                       
write.table(dst*13503, "./finished_genomes_SUD_13Feb2023_no-dates_aln_dist_matrix_raw23Feb2023.csv", quote=FALSE)


     -      a      b      c      g      k      m      n      r      s      t 
 23024 703962      1 484486 443491     27     33  22296     23     16 610867 
     w      y 
    26     28 


     a      c      g      t 
493609 366101 331394 429256 

120 DNA sequences in binary format stored in a matrix.

All sequences of same length: 19069 

Labels:
2022002242_C008
2022002244_CXX1
2022002246_C017
2022002247_C025
2022002248_C024
2022002249_C020
...

Base composition:
    a     c     g     t 
0.314 0.216 0.198 0.272 
(Total: 2.29 Mb)

120 DNA sequences in binary format stored in a matrix.

All sequences of same length: 13503 

Labels:
2022002242_C008
2022002244_CXX1
2022002246_C017
2022002247_C025
2022002248_C024
2022002249_C020
...

Base composition:
    a     c     g     t 
0.305 0.226 0.205 0.265 
(Total: 1.62 Mb)

,2022002242_C008,2022002244_CXX1,2022002246_C017,2022002247_C025,2022002248_C024,2022002249_C020,2022002255_C012,2022002258_C027,2022002261_C028,2022002270_C033,⋯,2022004120_C152,2022004191_C161,2022005176_C154,2022005178_CXX3,2022005180_C158,2022005181_C157,2022005182_C156,2022005183_C163,2022005185_C156,2022005193_C159
2022002242_C008,0,5,2,8,3,6,3,2,3,3,⋯,4,5,5,4,6,4,6,5,6,6
2022002244_CXX1,5,0,3,9,4,7,4,3,4,4,⋯,5,6,6,5,7,5,7,6,7,7
2022002246_C017,2,3,0,6,1,4,1,0,1,1,⋯,2,3,3,2,4,2,4,3,4,4
2022002247_C025,8,9,6,0,7,10,7,6,7,7,⋯,8,9,9,8,10,8,10,9,10,10
2022002248_C024,3,4,1,7,0,5,2,1,2,2,⋯,3,4,4,3,5,3,5,4,5,5
2022002249_C020,6,7,4,10,5,0,5,4,5,5,⋯,6,7,7,6,8,6,8,7,8,8
2022002255_C012,3,4,1,7,2,5,0,1,2,2,⋯,3,4,4,3,5,3,5,4,5,5
2022002258_C027,2,3,0,6,1,4,1,0,1,1,⋯,2,3,3,2,4,2,4,3,4,4
2022002261_C028,3,4,1,7,2,5,2,1,0,2,⋯,3,4,4,3,5,3,5,4,5,5
2022002270_C033,3,4,1,7,2,5,2,1,2,0,⋯,3,4,4,3,5,3,5,4,5,5


In [1]:
#Step 4: Convert genetic distance matrix into a contact list-like format.

#PYTHON CODE below.

#Final text file format:
#         id1       id2  distance
#0    MAN4194  MAN12490  0.003494
#0    MAN4194  MAN12541  0.000000

import pandas as pd
from itertools import combinations
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

data = pd.read_csv('./finished_genomes_SUD_13Feb2023_no-dates_aln_dist_matrix_raw23Feb2023.csv', sep='\s+', header=0, index_col=0, engine='python')

#print(data.head())

#Initialize the dataframe:
columns=['id1', 'id2', 'distance']
Microbe_trace_pairwise_edge_list_df = pd.DataFrame(columns=columns)
Microbe_trace_pairwise_edge_list_df = Microbe_trace_pairwise_edge_list_df.fillna(0) # with 0s rather than NaNs


row_counter=1
column_counter=1
temp_df = pd.DataFrame(columns=columns)

#Iterate through one side of the genetic distance triangle matrix and convert to contact list:
for label, row in data.iterrows():
    
    for column_counter in range(row_counter, len(row)): 
        
        temp_df = pd.DataFrame({'id1':label,
                        'id2':data.index[column_counter],
                        'distance':[row.iloc[column_counter]]})

        Microbe_trace_pairwise_edge_list_df=Microbe_trace_pairwise_edge_list_df.append(temp_df)
        column_counter=column_counter+1
        
    row_counter=row_counter+1

print(Microbe_trace_pairwise_edge_list_df)

#Write dataframe to text file.
Microbe_trace_pairwise_edge_list_df.to_csv(r'./finished_genomes_SUD_13Feb2023_no-dates_aln_dist_raw23Feb2023.csv', header=True, index=None, sep='\t', mode='w')

print("Done!")

                id1              id2 distance
0   2022002242_C008  2022002244_CXX1        5
0   2022002242_C008  2022002246_C017        2
0   2022002242_C008  2022002247_C025        8
0   2022002242_C008  2022002248_C024        3
0   2022002242_C008  2022002249_C020        6
..              ...              ...      ...
0   2022005182_C156  2022005185_C156        0
0   2022005182_C156  2022005193_C159        4
0   2022005183_C163  2022005185_C156        7
0   2022005183_C163  2022005193_C159        7
0   2022005185_C156  2022005193_C159        4

[7140 rows x 3 columns]
Done!


In [2]:
#Step 5: Convert final output to only epi identifiers.

#Change id1 and id2 values to use epi identifiers, instead of linked lab and epi identifiers, by matching a regular expression.

# Importing re package for using regular expressions 
import re 
import pandas as pd

#Read in data:
genetic_df = pd.read_csv('./finished_genomes_SUD_13Feb2023_no-dates_aln_dist_raw23Feb2023.csv', sep='\t')

print(genetic_df['id1'])

# Function to clean the names 
def Clean_names(Contact_name): 
    # Search for opening bracket in the name followed by 
    # any characters repeated any number of times 
    if re.search('.*_.*', Contact_name): 
  
        # Extract the position of the start of the capture group 
        # Return the cleaned name
        return re.search('.*_(.*)', Contact_name).group(1).lower() 
  
    else: 
        # if clean up needed return the same name 
        return Contact_name 
          
# Updated the id1/2 column values from:
#lab-MBK2594_epi-RDCEQT002164   to 
#RDCEQT002164
#to match the values in the epi_Df
        
genetic_df['id1'] = genetic_df['id1'].apply(Clean_names) 
genetic_df['id2'] = genetic_df['id2'].apply(Clean_names) 

#for column in genetic_df[['id1', 'id2']]:
#    # Select column contents by column name using [] operator
#    columnSeriesObj = genetic_df[column]
#    print('Colunm Name : ', column)
#    print('Column Contents : ', columnSeriesObj.values)
#    
#    for column_value in columnSeriesObj.values:
#        print(column_value)

print(genetic_df)

#Write dataframe to text file.
genetic_df.to_csv(r'./finished_genomes_SUD_13Feb2023_no-dates_aln_dist_raw_NewNames_23Feb2023.csv', header=True, index=None, sep='\t', mode='w')

print("Done!")

0       2022002242_C008
1       2022002242_C008
2       2022002242_C008
3       2022002242_C008
4       2022002242_C008
             ...       
7135    2022005182_C156
7136    2022005182_C156
7137    2022005183_C163
7138    2022005183_C163
7139    2022005185_C156
Name: id1, Length: 7140, dtype: object
       id1   id2  distance
0     c008  cxx1         5
1     c008  c017         2
2     c008  c025         8
3     c008  c024         3
4     c008  c020         6
...    ...   ...       ...
7135  c156  c156         0
7136  c156  c159         4
7137  c163  c156         7
7138  c163  c159         7
7139  c156  c159         4

[7140 rows x 3 columns]
Done!
